<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1LeagueScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Module

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
from contextlib import closing
from operator import attrgetter
from pprint import pprint
from subprocess import DEVNULL
from subprocess import Popen
import time


# Team Dic

In [3]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmpDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126','名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmp2Dic = {'清水':'126', '福岡':'135'}

J12021TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '横浜FC':'y-fc' , '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '徳島':'toku', '福岡':'fuku', '鳥栖':'tosu', '大分':'oita'}

def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

get_keys_from_value(TeamDic,'276')

['札幌']

<select id="comparison1_team"><option value="0">チーム選択</option><option value="276">札幌</option><option value="31219">八戸</option><option value="30673">岩手</option><option value="238">仙台</option><option value="30310">秋田</option><option value="294">山形</option><option value="30674">福島</option><option value="120">鹿島</option><option value="176">水戸</option><option value="30305">栃木</option><option value="30103">群馬</option><option value="122">浦和</option><option value="199">大宮</option><option value="121">千葉</option><option value="132">柏</option><option value="270">FC東京</option><option value="123">東京Ｖ</option><option value="30532">町田</option><option value="86">川崎Ｆ</option><option value="124">横浜FM</option><option value="296">横浜FC</option><option value="30675">YS横浜</option><option value="130">湘南</option><option value="30676">相模原</option><option value="150">甲府</option><option value="30528">松本</option><option value="30535">長野</option><option value="193">新潟</option><option value="30308">富山</option><option value="30526">金沢</option><option value="126">清水</option><option value="131">磐田</option><option value="30677">藤枝</option><option value="31296">沼津</option><option value="127">名古屋</option><option value="30302">岐阜</option><option value="134">京都</option><option value="128">Ｇ大阪</option><option value="133">Ｃ大阪</option><option value="136">神戸</option><option value="30000">鳥取</option><option value="30314">岡山</option><option value="129">広島</option><option value="30851">山口</option><option value="30536">讃岐</option><option value="30116">徳島</option><option value="30148">愛媛</option><option value="30974">今治</option><option value="135">福岡</option><option value="30313">北九州</option><option value="269">鳥栖</option><option value="30531">長崎</option><option value="30303">熊本</option><option value="207">大分</option><option value="31508">宮崎</option><option value="31042">鹿児島</option><option value="30317">琉球</option></select>

# ScrapeData

In [ ]:
def scrapeData(homeTeamID,awayTeamID,year):
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チーム毎の平均スタッツを取得
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0,2,4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)

    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'試合平均': 'home','Unnamed: 3': 'Data','試合平均.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    time.sleep(1)

    #試合結果を取得
    html = requests.get(url)

    html.encoding = "UTF-8"
    soup = BeautifulSoup(html.text, "html.parser")

    texts = soup.find('div', attrs={'class': 'gameTbl'}).text
    texts = re.findall(r'\w+', texts)
    time.sleep(1) 

    return df,texts

# Scrape Season Data

In [34]:
def scrapeSeasonData(TeamNameDic,dfList):
    for team in TeamNameDic:
        url = 'https://www.football-lab.jp/' +  TeamNameDic[team] + '/match/'
        #表を取得
        df = pd.read_html(url)[0]
        #最後の列を削除
        df = df.drop(38).drop(df.columns[[2,6,7,8,19,20]], axis=1)
        df.insert(3,'Team',team)
        dfList.append(df)
        time.sleep(1) 
    return dfList

In [35]:
tmpJ12021TeamDic = {'札幌':'sapp', '仙台':'send',}
dfList = []
dfList =  scrapeSeasonData(tmpJ12021TeamDic,dfList)
dfList[0]

,節,開催日,相手,Team,スコア,Unnamed: 5,AGI,KAGI,チャンス構築率,シュート,シュート成功率,支配率,攻撃CBP,パスCBP,奪取P,守備P
0,1,2.27,横浜FC,札幌,5-1,H,61,66,20.9%,23,21.7%,54.3%,24.11,18.98,131.07,11.56
1,2,3.6,名古屋,札幌,0-1,A,51,56,6.6%,8,0.0%,54.3%,15.23,13.46,102.32,9.16
2,3,3.10,広島,札幌,1-2,A,53,41,17.8%,21,4.8%,60.2%,20.30,15.49,83.07,12.25
3,5,3.17,浦和,札幌,0-0,A,56,55,16.9%,20,0.0%,58.5%,22.35,16.58,120.84,9.72
4,6,3.20,神戸,札幌,3-4,H,54,54,11.8%,15,20.0%,50.0%,19.43,13.50,112.56,15.64
5,7,4.3,福岡,札幌,2-1,A,55,49,9.4%,13,15.4%,56.2%,18.94,15.33,139.59,18.88
6,8,4.7,FC東京,札幌,1-2,A,64,58,16.2%,16,6.3%,57.9%,19.67,16.27,94.02,8.42
7,9,4.11,鹿島,札幌,2-2,H,53,45,14.4%,17,11.8%,56.4%,18.17,12.60,84.62,11.82
8,10,4.16,横浜FM,札幌,1-3,H,54,49,12.9%,17,5.9%,46.4%,14.97,11.51,128.30,13.12
9,11,4.24,仙台,札幌,2-1,H,52,46,14.3%,17,11.8%,65.7%,22.66,17.94,104.13,13.61


In [24]:
print(dfList[0])

None


In [ ]:
df = scrapeSeasonData(J12021TeamDic['川崎Ｆ'])
df